## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-22-03-54-08 +0000,bbc,High-profile Republican Marjorie Taylor Greene...,https://www.bbc.com/news/articles/cj4qx1lenvjo...
1,2025-11-22-03-46-35 +0000,nyt,Judge Places Hold on IRS Data Sharing With ICE,https://www.nytimes.com/2025/11/21/business/ir...
2,2025-11-22-03-43-22 +0000,startribune,Prep Bowl: Live updates from the football stat...,https://www.startribune.com/live-scores-and-ch...
3,2025-11-22-03-42-03 +0000,wapo,Grizzly bear attack on school group in Canada ...,https://www.washingtonpost.com/world/2025/11/2...
4,2025-11-22-03-36-00 +0000,wsj,"Republican Marjorie Taylor Greene, who fought ...",https://www.wsj.com/politics/policy/marjorie-t...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2344/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
8,trump,56
239,new,21
318,peace,17
448,ukraine,17
249,mamdani,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
43,2025-11-22-01-00-00 +0000,wsj,President Trump said this week he expects lowe...,https://www.wsj.com/economy/central-banking/fe...,167
144,2025-11-21-20-01-54 +0000,nyt,European Leaders Back Zelensky After Trump’s R...,https://www.nytimes.com/2025/11/21/world/europ...,127
79,2025-11-21-22-54-21 +0000,nyt,"As Trump Pushes on Russia-Ukraine Peace Plan, ...",https://www.nytimes.com/2025/11/21/world/europ...,125
63,2025-11-21-23-51-14 +0000,nypost,Zohran Mamdani’s stance during first Trump mee...,https://nypost.com/2025/11/21/us-news/trump-an...,121
112,2025-11-21-21-34-00 +0000,wsj,Trump Says He Wants Ukraine’s Answer on Peace ...,https://www.wsj.com/world/zelensky-russia-ukra...,121


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
43,167,2025-11-22-01-00-00 +0000,wsj,President Trump said this week he expects lowe...,https://www.wsj.com/economy/central-banking/fe...
65,77,2025-11-21-23-49-57 +0000,bbc,Zelensky warns Ukraine risks losing US support...,https://www.bbc.com/news/articles/cy5g4r9l4dlo...
35,62,2025-11-22-01-18-00 +0000,wsj,Texas asked the Supreme Court to let it use a ...,https://www.wsj.com/politics/elections/texas-a...
4,53,2025-11-22-03-36-00 +0000,wsj,"Republican Marjorie Taylor Greene, who fought ...",https://www.wsj.com/politics/policy/marjorie-t...
63,52,2025-11-21-23-51-14 +0000,nypost,Zohran Mamdani’s stance during first Trump mee...,https://nypost.com/2025/11/21/us-news/trump-an...
208,51,2025-11-21-16-11-09 +0000,bbc,"The ex-Reform UK leader who took bribes, made ...",https://www.bbc.com/news/videos/cx23l0jlwx2o?a...
343,41,2025-11-21-05-35-00 +0000,latimes,Race for California governor continues to heat...,https://www.latimes.com/california/story/2025-...
3,37,2025-11-22-03-42-03 +0000,wapo,Grizzly bear attack on school group in Canada ...,https://www.washingtonpost.com/world/2025/11/2...
146,35,2025-11-21-19-35-34 +0000,latimes,Suspect in car chase crash that killed an Alha...,https://www.latimes.com/california/story/2025-...
215,33,2025-11-21-15-53-00 +0000,wsj,The Democratic National Committee took out a $...,https://www.wsj.com/politics/policy/democratic...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
